In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

class PreprocesamientoDatos:
    def __init__(self):
        # Configurar stopwords
        self.stop_words = set(stopwords.words('english'))

        # Inicializar lematizador
        self.lemmatizer = WordNetLemmatizer()

    def limpiar_texto(self, texto):
        """
        Proceso completo de preprocesamiento de texto
        """
        if not isinstance(texto, str):
            return ""

        # 1. Convertir a minúsculas
        texto = texto.lower()

        # 2. Eliminación de caracteres especiales
        texto = re.sub(r'[^a-zA-Z\s]', '', texto)

        # 3. Tokenización
        tokens = word_tokenize(texto)

        # 4. Eliminación de stopwords
        tokens = [token for token in tokens if token not in self.stop_words]

        # 5. Lematización
        tokens_lematizados = [self.lemmatizer.lemmatize(token) for token in tokens]

        return ' '.join(tokens_lematizados)

class SistemaRecuperacion:
    def __init__(self, archivo_csv_criticas, archivo_csv_peliculas):
        self.archivo_csv_criticas = archivo_csv_criticas
        self.archivo_csv_peliculas = archivo_csv_peliculas
        self.reviews = self.cargar_datos(archivo_csv_criticas)
        self.movies = self.cargar_datos(archivo_csv_peliculas)
        self.preprocesador = PreprocesamientoDatos()
        self.vectorizador = TfidfVectorizer()

    def cargar_datos(self, archivo_csv):
        # Cargar datos desde el archivo CSV
        return pd.read_csv(archivo_csv)

    def preprocesar_criticas(self):
        # Preprocesar las críticas de las películas
        self.reviews['review_preprocessed'] = self.reviews['review_content'].apply(
            lambda texto: self.preprocesador.limpiar_texto(str(texto))
        )

    def indexar_criticas(self):
        """
        Indexar las críticas mediante un vector de características (TF-IDF)
        """
        # Usamos TF-IDF para convertir las críticas en vectores numéricos
        self.tfidf_matrix = self.vectorizador.fit_transform(self.reviews['review_preprocessed'])

    def procesar_consulta(self, consulta):
        """
        Procesar la consulta predefinida
        """
        # Preprocesar la consulta
        consulta_preprocesada = self.preprocesador.limpiar_texto(consulta)

        # Convertir la consulta a un vector usando el mismo vectorizador
        return self.vectorizador.transform([consulta_preprocesada])

    def recuperar_peliculas_relevantes(self, consulta_preprocesada):
        """
        Recuperar las películas más relevantes basadas en la consulta predefinida
        """
        # Calcular la similitud entre la consulta y las críticas
        similitudes = cosine_similarity(consulta_preprocesada, self.tfidf_matrix).flatten()

        # Ordenar las críticas por similitud (de mayor a menor)
        indices_similares = similitudes.argsort()[::-1]

        # Obtener las películas más relevantes
        peliculas_relevantes = self.reviews.iloc[indices_similares]

        # Mapear las películas a las críticas relevantes
        peliculas_relevantes = peliculas_relevantes.merge(self.movies,
                                                          left_on="rotten_tomatoes_link",
                                                          right_on="rotten_tomatoes_link")

        # Seleccionar solo las columnas relevantes (título y link)
        peliculas_relevantes = peliculas_relevantes[['movie_title', 'rotten_tomatoes_link', 'review_content']]

        return peliculas_relevantes

# Ejemplo de uso
def main():
    # Definir las rutas de los archivos CSV
    archivo_criticas = 'rotten_tomatoes_critic_reviews.csv'
    archivo_peliculas = 'rotten_tomatoes_movies.csv'

    # Iniciar el sistema de recuperación
    sistema = SistemaRecuperacion(archivo_criticas, archivo_peliculas)

    # Preprocesar las críticas
    sistema.preprocesar_criticas()

    # Indexar las críticas
    sistema.indexar_criticas()

    # Definir la consulta predefinida (por ejemplo, una consulta de fantasía y aventura)
    consulta = "A fantasy adventure movie"

    # Procesar la consulta
    consulta_preprocesada = sistema.procesar_consulta(consulta)

    # Recuperar las películas más relevantes
    peliculas_relevantes = sistema.recuperar_peliculas_relevantes(consulta_preprocesada)

    # Mostrar las películas relevantes
    print("\nPelículas más relevantes:")
    print(peliculas_relevantes.head(10))  # Mostrar las primeras 10 películas

if __name__ == '__main__':
    main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
